In [1]:
import pickle
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import operator

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, recall_score, precision_score
from bayes_opt import BayesianOptimization

from general_functions import create_balanced_dataset

In [2]:
file = open("dataset/zone_4.pickle", "rb")
zone_4 = pickle.load(file)

file = open("dataset/zone_7.pickle", "rb")
zone_7 = pickle.load(file)
file = None

In [3]:
with open("dataset/zone_4_resampled.pickle", "rb") as file:
    zone_4_resampled = pickle.load(file)
with open("dataset/zone_7_resampled.pickle", "rb") as file:
    zone_7_resampled = pickle.load(file)

In [4]:
experiment_arr = [(zone_4_resampled, zone_7), (zone_7_resampled, zone_4)]

In [5]:
zone_4 = None
zone_7 = None
zone_4_resampled = None
zone_7_resampled = None

In [6]:
features_to_use = [
    ['hpmf_raw', 'skyview_raw', 'impundment_raw', 'slope_raw', 'DEM_ditch_detection', 'DEM_ditch_detection_no_streams',
     'conic_mean', 'skyview_non_ditch', 'skyview_gabor', 'conic_mean_no_streams', 'skyview_gabor_no_streams',
     'skyview_mean_2', 'skyview_mean_3', 'skyview_mean_4', 'skyview_mean_6', 'skyview_median_2', 'skyview_median_4',
     'skyview_median_6', 'skyview_min_2', 'skyview_min_4', 'skyview_min_6', 'skyview_max_2', 'skyview_max_4',
     'skyview_max_6', 'skyview_std_2', 'skyview_std_4', 'skyview_std_6', 'impundment_amplified',
     'impoundment_amplified_no_streams', 'impundment_mean_2', 'impundment_mean_3', 'impundment_mean_4',
     'impundment_mean_6', 'impundment_median_2', 'impundment_median_4', 'impundment_median_6', 'impundment_min_2',
     'impundment_min_4', 'impundment_min_6', 'impundment_max_2', 'impundment_max_4', 'impundment_max_6',
     'impundment_std_2', 'impundment_std_4', 'impundment_std_6', 'hpmf_filter', 'hpmf_gabor', 'hpmf_gabor_no_streams',
     'hpmf_filter_no_streams', 'hpmf_mean_2', 'hpmf_mean_3', 'hpmf_mean_4', 'hpmf_mean_6', 'hpmf_median_2',
     'hpmf_median_4', 'hpmf_median_6', 'hpmf_min_2', 'hpmf_min_4', 'hpmf_min_6', 'hpmf_max_2', 'hpmf_max_4',
     'hpmf_max_6', 'hpmf_std_2', 'hpmf_std_4', 'hpmf_std_6', 'slope_non_ditch', 'slope_mean_2', 'slope_mean_3',
     'slope_mean_4', 'slope_mean_6', 'slope_median_2', 'slope_median_4', 'slope_median_6', 'slope_min_2', 'slope_min_4',
     'slope_min_6', 'slope_max_2', 'slope_max_4', 'slope_std_2', 'slope_std_4', 'slope_std_6'],
    ['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2', 'hpmf_mean_4', 'hpmf_mean_3',
     'impundment_amplified', 'hpmf_median_4', 'impoundment_amplified_no_streams', 'skyview_max_6', 'skyview_gabor',
     'hpmf_min_2', 'impundment_max_2', 'impundment_std_4', 'skyview_non_ditch', 'skyview_gabor_no_streams', 'slope_min_6',
     'skyview_max_4', 'impundment_median_2', 'impundment_max_6', 'slope_non_ditch', 'impundment_mean_6', 'hpmf_filter',
     'impundment_std_6', 'hpmf_filter_no_streams', 'impundment_median_6', 'hpmf_mean_6', 'slope_min_4', 'slope_std_6',
     'hpmf_mean_2', 'slope_median_6', 'impundment_max_4', 'impundment_std_2', 'hpmf_gabor', 'hpmf_min_4', 'skyview_min_6',
     'skyview_median_6', 'slope_mean_6', 'hpmf_gabor_no_streams', 'skyview_std_6', 'slope_min_2', 'hpmf_median_2',
     'skyview_max_2', 'hpmf_std_6', 'slope_std_4', 'hpmf_min_6', 'slope_max_4', 'skyview_mean_6', 'hpmf_max_6', 'skyview_min_4'],
    ['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2', 'impundment_amplified',
     'impoundment_amplified_no_streams', 'hpmf_median_4', 'skyview_max_6', 'hpmf_mean_4', 'skyview_gabor',
     'skyview_non_ditch', 'slope_min_6', 'skyview_max_4', 'skyview_gabor_no_streams', 'slope_median_6', 'impundment_max_6',
     'hpmf_filter', 'skyview_median_6', 'slope_non_ditch', 'impundment_std_4', 'slope_std_6', 'hpmf_filter_no_streams',
     'impundment_median_6', 'hpmf_mean_6', 'slope_min_4', 'hpmf_mean_3', 'impundment_mean_6', 'slope_min_2', 'slope_mean_6',
     'skyview_max_2', 'slope_std_4', 'impundment_median_2', 'hpmf_std_6', 'skyview_min_6', 'impundment_std_6'],
    ['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_amplified', 'hpmf_median_4',
     'impoundment_amplified_no_streams', 'impundment_mean_2', 'skyview_max_6', 'skyview_gabor', 'skyview_non_ditch',
     'slope_min_6', 'hpmf_mean_6', 'skyview_median_6', 'skyview_max_4', 'skyview_min_6', 'impundment_max_6', 'slope_std_6',
     'slope_median_6', 'skyview_gabor_no_streams', 'hpmf_std_6', 'impundment_std_4', 'slope_std_4', 'hpmf_filter',
     'impundment_median_6', 'slope_mean_6'],
    ['impundment_mean_3', 'hpmf_median_4', 'impundment_mean_4', 'impundment_amplified', 'skyview_non_ditch', 'skyview_max_6',
     'hpmf_filter', 'impoundment_amplified_no_streams', 'hpmf_mean_6', 'slope_min_6', 'impundment_max_6', 'skyview_median_6',
     'skyview_gabor', 'skyview_min_6', 'skyview_max_4', 'slope_median_6', 'impundment_mean_2', 'impundment_median_6',
     'hpmf_std_6', 'slope_std_6'],
    ['impundment_mean_3', 'impundment_mean_4', 'hpmf_median_4', 'skyview_gabor', 'impundment_amplified', 'skyview_max_6',
     'skyview_non_ditch', 'impundment_max_6', 'hpmf_filter', 'impoundment_amplified_no_streams', 'slope_min_6',
     'hpmf_mean_6', 'slope_median_6', 'slope_std_6', 'skyview_median_6'],
    ['impundment_mean_3', 'impundment_mean_4', 'skyview_max_6', 'hpmf_median_4', 'skyview_gabor', 'impundment_max_6',
     'impundment_amplified', 'hpmf_filter', 'slope_min_6', 'skyview_non_ditch']
]

In [7]:
for most_important_features in features_to_use:
    num_features = len(most_important_features)
    print(f"Features used in experiment:\n{most_important_features}")
    print("\n")
    y_test_all = []
    pred_all = []
    
    for (training_dataset, test_dataset) in experiment_arr:

        X_train = training_dataset.filter(items=most_important_features).loc[:, training_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_train = training_dataset["label_3m"]
        training_dataset = None
        
        
        clf = make_pipeline(StandardScaler(), LinearSVC(C=4.228518,
                                                        tol=0.18626,
                                                        dual=False,
                                                        fit_intercept=False,
                                                        class_weight="balanced",
                                                        intercept_scaling=1.46756,
                                                        max_iter=471,
                                                        verbose=3))

        clf.fit(X_train, y_train)

        X_test = test_dataset.filter(items=most_important_features).loc[:, test_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_test = test_dataset["label_3m"]
        
        print(f"Amount of features X_train: {len(X_train.columns)}, X_test: {len(X_test.columns)}")

        pred = clf.predict(X_test)

        for i, pred_var in enumerate(pred):
            y_test_all.append(y_test[i])
            pred_all.append(pred_var)

    print("\n")
    print(f"Amount of features used:   {num_features}")
    print("Cohen's kappa score        ", cohen_kappa_score(y_test_all, pred_all))
    print("Accuracy score             ", accuracy_score(y_test_all, pred_all))
    print("Recall score               ", recall_score(y_test_all, pred_all))
    print("Precision score            ", precision_score(y_test_all, pred_all))
    print("\n------------------------------------------------------\n")

Features used in experiment:
['hpmf_raw', 'skyview_raw', 'impundment_raw', 'slope_raw', 'DEM_ditch_detection', 'DEM_ditch_detection_no_streams', 'conic_mean', 'skyview_non_ditch', 'skyview_gabor', 'conic_mean_no_streams', 'skyview_gabor_no_streams', 'skyview_mean_2', 'skyview_mean_3', 'skyview_mean_4', 'skyview_mean_6', 'skyview_median_2', 'skyview_median_4', 'skyview_median_6', 'skyview_min_2', 'skyview_min_4', 'skyview_min_6', 'skyview_max_2', 'skyview_max_4', 'skyview_max_6', 'skyview_std_2', 'skyview_std_4', 'skyview_std_6', 'impundment_amplified', 'impoundment_amplified_no_streams', 'impundment_mean_2', 'impundment_mean_3', 'impundment_mean_4', 'impundment_mean_6', 'impundment_median_2', 'impundment_median_4', 'impundment_median_6', 'impundment_min_2', 'impundment_min_4', 'impundment_min_6', 'impundment_max_2', 'impundment_max_4', 'impundment_max_6', 'impundment_std_2', 'impundment_std_4', 'impundment_std_6', 'hpmf_filter', 'hpmf_gabor', 'hpmf_gabor_no_streams', 'hpmf_filter_no_st

# Optimization

In [18]:
def optim_function(tol=0.0001,
                   C=1,
                   intercept_scaling=1,
                   verbose=0,
                   max_iter=1000,
                   class_weight=0,
                   fit_intercept = 0,
                   dual = 0):
    
    max_iter = int(max_iter)
    verbose = int(verbose)
    class_weight = None if int(class_weight) == 0 else "balanced"
    fit_intercept = False if int(fit_intercept) == 0 else True
    dual = False if int(dual) == 0 else True
    
    clf = make_pipeline(StandardScaler(), LinearSVC(tol=tol,
                                                    C=C,
                                                    intercept_scaling=intercept_scaling,
                                                    verbose=verbose,
                                                    max_iter=max_iter,
                                                    class_weight=class_weight,
                                                    fit_intercept=fit_intercept,
                                                    dual=dual))

    clf.fit(X_train, y_train)
    
    pred = clf.predict(X_test)
    kappa = cohen_kappa_score(y_test, pred)
    return kappa

In [5]:
X_train, y_train = np.array(zone_4_resampled.loc[:, zone_4_resampled.columns != 'label_3m']), np.array(zone_4_resampled["label_3m"]).astype(int)
zone_4_resampled = None
zone_7_resampled = None

X_test, y_test = np.array(zone_7.loc[:, zone_7.columns != 'label_3m']), np.array(zone_7["label_3m"]).astype(int)
zone_7 = None
zone_4 = None

In [15]:
X_train.shape

(317175, 81)

In [19]:
pbounds = {"tol": (0.000001, 1),
           "C": (0.1, 10),
           "intercept_scaling": (0, 10),
           "verbose": (0, 10),
           "max_iter": (10, 5000),
           "class_weight": (0, 2),
           "fit_intercept": (0, 2),
           "dual": (0, 2)
          }

optimizer = BayesianOptimization(
    f=optim_function,
    pbounds=pbounds,
    random_state=1,
    verbose=2,
)

In [20]:
optimizer.maximize(n_iter=100)

|   iter    |  target   |     C     | class_... |   dual    | fit_in... | interc... | max_iter  |    tol    |  verbose  |
-------------------------------------------------------------------------------------------------------------------------
[LibLinear]|  1        |  0.4329   |  4.229    |  1.441    |  0.000228 |  0.6047   |  1.468    |  470.8    |  0.1863   |  3.456    |
[LibLinear]|  2        |  0.3304   |  4.028    |  1.078    |  0.8384   |  1.37     |  2.045    |  4.392e+0 |  0.02739  |  6.705    |
[LibLinear]|  3        |  0.3801   |  4.231    |  1.117    |  0.2808   |  0.3962   |  8.007    |  4.842e+0 |  0.3134   |  6.923    |
[LibLinear]|  4        |  0.4329   |  8.776    |  1.789    |  0.1701   |  0.07811  |  1.698    |  4.392e+0 |  0.09835  |  4.211    |
[LibLinear]

C:\Users\Jonatan\.conda\envs\pytorch\lib\site-packages\sklearn\svm\_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


|  5        |  0.2675   |  9.583    |  1.066    |  1.384    |  0.631    |  6.865    |  4.175e+0 |  0.01829  |  7.501    |
|  6        |  0.3801   |  10.0     |  2.0      |  0.0      |  0.0      |  0.0      |  4.595e+0 |  1.0      |  0.0      |
[LibLinear]|  7        |  0.4329   |  3.787    |  1.789    |  0.2342   |  0.6831   |  4.834    |  689.0    |  0.1647   |  1.741    |
[LibLinear]

C:\Users\Jonatan\.conda\envs\pytorch\lib\site-packages\sklearn\svm\_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


|  8        |  0.1008   |  3.822    |  1.538    |  1.218    |  0.03645  |  6.874    |  245.1    |  0.1495   |  6.514    |
[LibLinear]

...

C:\Users\Jonatan\.conda\envs\pytorch\lib\site-packages\sklearn\svm\_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


|  46       |  0.2781   |  4.585    |  0.2227   |  1.613    |  1.879    |  1.803    |  3.37e+03 |  0.7787   |  8.015    |
[LibLinear]

C:\Users\Jonatan\.conda\envs\pytorch\lib\site-packages\sklearn\svm\_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


|  47       |  0.2773   |  1.474    |  1.722    |  1.25     |  0.02928  |  3.886    |  1.071e+0 |  0.4483   |  9.598    |
|  48       |  0.3791   |  9.585    |  0.4678   |  0.4763   |  0.5932   |  3.091    |  3.534e+0 |  0.6572   |  0.04504  |
[LibLinear]

KeyboardInterrupt: 

In [24]:
optimizer.max

{'target': 0.4328530799039061,
 'params': {'C': 4.228517846555483,
  'class_weight': 1.4406489868843162,
  'dual': 0.00022874963468977327,
  'fit_intercept': 0.6046651452636795,
  'intercept_scaling': 1.4675589081711304,
  'max_iter': 470.769587896301,
  'tol': 0.1862610251174595,
  'verbose': 3.4556072704304777}}